# RiskLab: interactive financial risk assessment toolkit

this notebook demonstrates the core functionality of **RiskLab**, a toolkit designed to analyze and visualize portfolio risks  

the goals of this project are:

- **Portfolio Analysis**: load a portfolio of assets with their weights and attributes

- **Risk Metrics**: compute common measures such as volatility, Value-at-Risk (VaR), Conditional VaR, and sharpe ratio 

- **Stress Testing**: simulate different market scenarios (e.g., crashes, rallies) and assess portfolio performance under stress

- **Liquidity Metrics**: evaluate how liquidity conditions affect portfolio stability

- **Risk Matrix**: visualize the relationship between risk likelihoods and severities in a heatmap for intuitive understanding  

unlike the streamlit dashboard, this notebook focuses on **step-by-step testing and validation** of the underlying computations

each section corresponds to a different aspect of financial risk analysis and can be run independently

In [3]:
import sys
import pandas as pd
import numpy as np
import plotly.express as px

sys.path.append("../src")

from portfolio import Portfolio
from risk_metrics import RiskMetrics
from stress_test import StressTest
from liquidity import LiquidityMetrics
from risk_matrix import RiskMatrix

pd.set_option("display.precision", 4)
pd.set_option("display.max_columns", None)

print("Libraries and modules imported successfully.")

Libraries and modules imported successfully.


In [4]:
portfolio = Portfolio.from_csv("../data/sample_portfolio.csv")

print("Portfolio loaded successfully. Number of tickers:", len(portfolio.tickers))
portfolio.data

Portfolio loaded successfully. Number of tickers: 8


,Ticker,Weight,AssetType
0,AAPL,0.20,Equity
1,MSFT,0.20,Equity
2,TSLA,0.15,Equity
3,AMZN,0.15,Equity
4,GOOG,0.10,Equity
5,NVDA,0.10,Equity
6,JPM,0.05,Bond
7,XOM,0.05,Commodity


In [5]:
np.random.seed(42)

returns = pd.DataFrame(
    np.random.normal(0, 0.01, (252, len(portfolio.tickers))),
    columns=portfolio.tickers
)

print("Dummy returns generated successfully. Shape:", returns.shape)
returns.head()

Dummy returns generated successfully. Shape: (252, 8)


,AAPL,MSFT,TSLA,AMZN,GOOG,NVDA,JPM,XOM
0,0.0050,-0.0014,0.0065,0.0152,-0.0023,-0.0023,0.0158,0.0077
1,-0.0047,0.0054,-0.0046,-0.0047,0.0024,-0.0191,-0.0172,-0.0056
2,-0.0101,0.0031,-0.0091,-0.0141,0.0147,-0.0023,0.0007,-0.0142
3,-0.0054,0.0011,-0.0115,0.0038,-0.0060,-0.0029,-0.0060,0.0185
4,-0.0001,-0.0106,0.0082,-0.0122,0.0021,-0.0196,-0.0133,0.0020


In [6]:
rm = RiskMetrics(portfolio, returns)

rm.compute_volatility()
rm.compute_var()
rm.compute_cvar()
rm.compute_sharpe()

metrics_numeric = rm.summary(formatted=False)
metrics_display = rm.summary(formatted=True)

metrics_df = pd.DataFrame([metrics_numeric], index=["Base"])
metrics_display_df = pd.DataFrame([metrics_display], index=["Base"])

print("Risk metrics computed successfully.")
metrics_display_df

Risk metrics computed successfully.


,Volatility,VaR_95,CVaR_95,Sharpe
Base,0.36%,0.52%,0.62%,0.1494


In [7]:
st_test = StressTest(portfolio, returns)

scenarios = {
    "Base": {t: 0.0 for t in portfolio.tickers},
    "Market Crash": {t: -0.1 for t in portfolio.tickers},
    "Tech Dip": {t: -0.15 for t in portfolio.tickers},
    "Bond Rally": {t: 0.05 for t in portfolio.tickers},
}

for name, adjustments in scenarios.items():
    st_test.apply_scenario(name, adjustments)

scenario_df = st_test.summary().copy()
scenario_df.reset_index(inplace=True)
scenario_df.rename(columns={scenario_df.columns[0]: "Scenario"}, inplace=True)

print("Stress test scenarios applied successfully.")
display(scenario_df)

scenario_df_plot = scenario_df.melt(id_vars="Scenario", var_name="Metric", value_name="Value")
fig = px.line(
    scenario_df_plot,
    x="Scenario",
    y="Value",
    color="Metric",
    markers=True,
    title="Portfolio Stress Test Results"
)
fig.show()

Stress test scenarios applied successfully.


,Scenario,Volatility,VaR_95,CVaR_95,Sharpe
0,Base,0.36%,0.52%,0.62%,0.1494
1,Market Crash,0.32%,0.47%,0.55%,0.1494
2,Tech Dip,0.30%,0.45%,0.52%,0.1494
3,Bond Rally,0.38%,0.55%,0.65%,0.1494


In [9]:
scores = {t: 0.8 for t in portfolio.tickers}  
lm = LiquidityMetrics(portfolio, scores)

lm.apply_scenario("Liquidity Crunch", {"Equity": -0.2, "Bond": -0.05})
lm.apply_scenario("Tech Rally", {"Equity": 0.1, "Bond": 0.02})

liq_df = lm.summary().reset_index()
liq_df.rename(columns={liq_df.columns[0]: "Scenario"}, inplace=True)

print("Liquidity scenarios applied successfully.")
display(liq_df)

fig_liq = px.bar(
    liq_df,
    x="Scenario",
    y="Portfolio Liquidity",
    text="Portfolio Liquidity",
    color="Portfolio Liquidity",
    color_continuous_scale="Blues",
    title="liquidity metrics across scenarios"
)
fig_liq.show()

Liquidity scenarios applied successfully.


,Scenario,Portfolio Liquidity
0,Liquidity Crunch,0.6540
1,Tech Rally,0.8728


In [10]:
rmat = RiskMatrix(portfolio, rm, {t: 0.1 + 0.1*np.random.rand() for t in portfolio.tickers})
rm_df = rmat.compute_matrix().copy()
rm_df.columns = rm_df.columns.str.strip()

if "RiskScore" in rm_df.columns:
    rm_values = rm_df["RiskScore"].values.reshape(-1, 1)
    labels = rm_df.index.tolist()
    x_labels = ["RiskScore"]
elif rm_df.shape[0] == rm_df.shape[1]:
    rm_values = rm_df.values
    labels = rm_df.index.tolist()
    x_labels = rm_df.columns
else:
    print("Unexpected RiskMatrix format. Using dummy zeros.")
    rm_df["RiskScore"] = np.zeros(len(rm_df))
    rm_values = rm_df["RiskScore"].values.reshape(-1, 1)
    labels = rm_df.index.tolist()
    x_labels = ["RiskScore"]

fig_heat = px.imshow(
    rm_values,
    x=x_labels,
    y=labels,
    labels=dict(x="Ticker", y="Ticker", color="Risk Score"),
    color_continuous_scale="RdYlGn_r",
    text_auto=".4f",
    title="Risk Matrix Heatmap"
)
fig_heat.update_layout(width=800, height=600)
fig_heat.show()

In [12]:
summary_data = {
    "Volatility": [metrics_display["Volatility"]],
    "VaR 95%": [metrics_display["VaR_95"]],
    "CVaR 95%": [metrics_display["CVaR_95"]],
    "Sharpe Ratio": [metrics_display["Sharpe"]],
    "Liquidity Score (Latest)": [liq_df["Portfolio Liquidity"].iloc[-1]]
}

summary_df = pd.DataFrame(summary_data, index=["Base Portfolio"])

print("Portfolio Summary Dashboard:")
display(summary_df.style.background_gradient(cmap="RdYlGn", axis=1))

Portfolio Summary Dashboard:


,Volatility,VaR 95%,CVaR 95%,Sharpe Ratio,Liquidity Score (Latest)
Base Portfolio,0.36%,0.52%,0.62%,0.149400,0.872800


# RISKLAB NOTEBOOK COMPLETE

>KEEP THIS IN MIND: "THE GAME'S NEVER REALLY OVER… YOU JUST KEEP RUNNING"

ALL ANALYSES HAVE BEEN EXECUTED: PORTFOLIO OVERVIEW, RISK METRICS, STRESS TESTS, LIQUIDITY SCENARIOS, AND THE RISK MATRIX HEATMAP